### Importing main API and Data Source

In [368]:
import numpy as np
import calendar as cal
from datetime import datetime
import matplotlib.pyplot as plt

### Task (1)

In [369]:
# TODO : Calculate the overall average temperature.
DATA = "wetter.csv"
# Reading CSV file and transforming it to full data array.
arr_total = np.genfromtxt(DATA, delimiter=',', skip_header=1)   # dtype=float by default

# Putting Data into a 1D Array (data = [1, 2, 3, 4, 5, 6, 9.87654321])
data = arr_total[:,2]                 # (:) all rows, (2) 3rd column from the left (count starts at 0)
mean_temp_all = np.mean(data)            # averaging dataset with the 'np.mean()' function

# Printing shapes/results for reference.
print(f"shape of 'arr_total': {arr_total.shape}\n---\n")
print(f"average temp (all year): {mean_temp_all:.2f}°C")

shape of 'arr_total': (2601, 5)
---

average temp (all year): 12.10°C


### Task (2) & (3)

In [391]:
# TODO : Calculate the average temperature for the month of July.
# TODO : Compare whether the months of July and May differ significantly in their average temperature

# Reading CSV file and transforming it to full data array. Keeping Date column as string -> Tuple.
arr = np.genfromtxt(DATA, delimiter=',', skip_header=1, dtype=None, encoding='utf-8')

def mean_temp_func(ls, month):
    mm = {f"{int(m):02d}" for m in month}
    vals = [float(r[2]) for r in ls if isinstance(r[0], str) and r[0][5:7] in mm]
    return float(np.nanmean(vals)) if vals else np.nan

# Printing average temp
print(f"# Task(2):      Average temp (jul): {mean_temp_func(arr,{7}):.2f}°C")
print(f"# Task(2):      Average temp (may): {mean_temp_func(arr,{5}):.2f}°C")
print(f"# Task(3):      Difference temp (ΔT[may,jul]): {np.abs(mean_temp_func(arr,{5})-mean_temp_func(arr,{7})):.2f}°C\n")
print("-"*55 + "\n")
print(f"# Task(3.1):    Difference temp (ΔT[apr,jun]): {np.abs(mean_temp_func(arr,{4})-mean_temp_func(arr,{6})):.2f}°C")
print(f"# Task(3.2):    Difference temp (ΔT[mar,may]): {np.abs(mean_temp_func(arr,{3})-mean_temp_func(arr,{5})):.2f}°C")
print(f"# Task(3.3):    Difference temp (ΔT[feb,apr]): {np.abs(mean_temp_func(arr,{2})-mean_temp_func(arr,{4})):.2f}°C")
print(f"# Task(3.4):    Difference temp (ΔT[jan,mar]): {np.abs(mean_temp_func(arr,{1})-mean_temp_func(arr,{3})):.2f}°C")
print(f"# Task(3.5):    Difference temp (ΔT[dec,feb]): {np.abs(mean_temp_func(arr,{12})-mean_temp_func(arr,{2})):.2f}°C")
print(f"# Task(3.6):    Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{11})-mean_temp_func(arr,{1})):.2f}°C")
print(f"# Task(3.7):    Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{10})-mean_temp_func(arr,{12})):.2f}°C")
print(f"# Task(3.8):    Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{9})-mean_temp_func(arr,{11})):.2f}°C")
print(f"# Task(3.9):    Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{8})-mean_temp_func(arr,{10})):.2f}°C")
print(f"# Task(3.10):   Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{7})-mean_temp_func(arr,{9})):.2f}°C")
print(f"# Task(3.11):   Difference temp (ΔT[nov,jan]): {np.abs(mean_temp_func(arr,{6})-mean_temp_func(arr,{8})):.2f}°C\n")
print("-"*55 + "\n")
print(f"# Average temp (ΔT[may,jun,jul]): {mean_temp_func(arr,{5,6,7}):.2f}°C")

# Task(2):      Average temp (jul): 20.75°C
# Task(2):      Average temp (may): 15.24°C
# Task(3):      Difference temp (ΔT[may,jul]): 5.51°C

-------------------------------------------------------

# Task(3.1):    Difference temp (ΔT[apr,jun]): 7.80°C
# Task(3.2):    Difference temp (ΔT[mar,may]): 7.88°C
# Task(3.3):    Difference temp (ΔT[feb,apr]): 6.80°C
# Task(3.4):    Difference temp (ΔT[jan,mar]): 4.50°C
# Task(3.5):    Difference temp (ΔT[dec,feb]): 1.22°C
# Task(3.6):    Difference temp (ΔT[nov,jan]): 4.53°C
# Task(3.7):    Difference temp (ΔT[nov,jan]): 7.52°C
# Task(3.8):    Difference temp (ΔT[nov,jan]): 9.55°C
# Task(3.9):    Difference temp (ΔT[nov,jan]): 7.67°C
# Task(3.10):   Difference temp (ΔT[nov,jan]): 3.81°C
# Task(3.11):   Difference temp (ΔT[nov,jan]): 1.80°C

-------------------------------------------------------

# Average temp (ΔT[may,jun,jul]): 18.15°C


### All results from Task (1), (2) & (3)

In [371]:
# Results printed!

print("\n" + "-"*55)
print(" TEMPERATURE STATISTICS")
print("-"*55 + "\n")

print(f"mean_T (all)        = {mean_temp_all:6.2f}°C")
print(f"mean_T (may-july)   = {mean_temp_func(arr,{5,6,7}):6.2f}°C\n")

print(f"mean_T (may)        = {mean_temp_func(arr,{5}):6.2f}°C")
print(f"mean_T (july)       = {mean_temp_func(arr,{7}):6.2f}°C\n")

print(f"ΔT(may, july)       = |{mean_temp_func(arr,{5}):.2f}°C - {mean_temp_func(arr,{7}):.2f}°C| = {np.abs(mean_temp_func(arr,{5})-mean_temp_func(arr,{7})):.2f}°C\n")

print("-"*55 + "\n")


-------------------------------------------------------
 TEMPERATURE STATISTICS
-------------------------------------------------------

mean_T (all)        =  12.10°C
mean_T (may-july)   =  18.15°C

mean_T (may)        =  15.24°C
mean_T (july)       =  20.75°C

ΔT(may, july)       = |15.24°C - 20.75°C| = 5.51°C

-------------------------------------------------------



# ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
## Just for fun...

In [372]:

# Extracting and Converting first column[0] into actual date/time.
dates = np.array([datetime.strptime(row[0], "%Y-%m-%d") for row in arr])

# Extracting temperature column.
temps = np.array([row[2] for row in arr], dtype=float)

# Printing shapes for reference.
print(f"Shape of 'arr': {arr.shape}")
print(f"Shape of 'dates': {dates.shape}")
print(f"Shape of 'temps': {temps.shape}")

Shape of 'arr': (2601,)
Shape of 'dates': (2601,)
Shape of 'temps': (2601,)


In [373]:
# create months once
months_num = np.array([d.month for d in dates])
print(f"Shape of 'months_num': {months_num.shape}")

# ---------
# define mask specs as a list to feed into calculations (you can add more tuples/lists)
masked_months_num = [[5, 6, 7], [5], [7]]
# build list of boolean masks (each mask is 1D, length == len(dates))
masks = [np.isin(months_num, vals) for vals in masked_months_num]
# filtered temperatures per mask: list of 1D numpy arrays
temps_filtered = [temps[m] for m in masks]
# convert to object array if you prefer numpy-indexable container
temps_filtered_arr = np.array(temps_filtered, dtype=object)
print(f"Shape of 'temps_filtered_arr': {temps_filtered_arr.shape}")
# ---------

# per-mask means (use nanmean to ignore NaNs)
mean_temps = np.array([np.nanmean(i) if i.size else np.nan for i in temps_filtered])
print(f"Shape of 'mean_temps': {mean_temps.shape}")


Shape of 'months_num': (2601,)
Shape of 'temps_filtered_arr': (3,)
Shape of 'mean_temps': (3,)


In [374]:
# INFORMEL :  Introducing 'no.abs()' & showing the filter work.

# calculation (MAE [Mean Absolut Error] – also is a good use case for the np.abs() function!)
diff = np.abs(mean_temps[1]-mean_temps[2])

# example prints by integer index
## print("temps_filtered for mask 0:", temps_filtered_arr[0])   # array of temps for [5,7]

In [375]:
# TODO : Calculate the average temperature for the month of July.

# Reading CSV file and transforming it to full data array. Keeping Date column as string -> Tuple.
arr = np.genfromtxt(DATA, delimiter=',', skip_header=1, dtype=None, encoding='utf-8')

def month_to_num(s):
    s = str(s).strip()
    for fmt in ('%b', '%B'): # 'Jul' or 'July'
        try:
            return datetime.strptime(s, fmt).month
        except ValueError:
            pass
    raise ValueError(f"Unrecognized month: {s}")

def mean_temp_func(ls, month):
    mm = {f"{int(m):02d}" for m in month}
    vals = [float(r[2]) for r in ls if isinstance(r[0], str) and r[0][5:7] in mm]
    return float(np.nanmean(vals)) if vals else np.nan

# Printing average temp

print(f"average temp (july): {mean_temp_func(arr,{'5','7'}):.2f}°C")
print(f"month_to_num test: {month_to_num('may','jul')}")

average temp (july): 17.99°C


TypeError: month_to_num() takes 1 positional argument but 2 were given

### Organizing Final Outputs

In [ ]:
# Results printed!

print("\n" + "-"*55)
print(" TEMPERATURE STATISTICS")
print("-"*55 + "\n")

print(f"mean_T (all)        = {mean:6.2f}°C")
print(f"mean_T (may-july)   = {mean_temps[0]:6.2f}°C\n")

print(f"mean_T (may)        = {mean_temps[1]:6.2f}°C")
print(f"mean_T (july)       = {mean_temps[2]:6.2f}°C\n")

print(f"ΔT(may, july)       = |{mean_temps[1]:.2f}°C - {mean_temps[2]:.2f}°C| = {np.abs(mean_temps[1]-mean_temps[2]):.2f}°C\n")
print(f"data                = {data}\n")

print("-"*55 + "\n")


-------------------------------------------------------
 TEMPERATURE STATISTICS
-------------------------------------------------------

mean_T (all)        =  12.10°C
mean_T (may-july)   =  18.15°C

mean_T (may)        =  15.24°C
mean_T (july)       =  20.75°C

ΔT(may, july)       = |15.24°C - 20.75°C| = 5.51°C

data                = [ 9.825   7.4375  5.5375 ... 20.7375 20.45   21.0625]

-------------------------------------------------------

